In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
train = pd.DataFrame({"path":glob.glob("/kaggle/input/plant-seedlings-classification/train/*/*")})

In [ ]:
train

In [ ]:
train["path"][0]

In [ ]:
# pd.options.display.max_colwidth = 999

In [ ]:
train["label"] = train["path"].apply(lambda x : x.split("/")[5])

In [ ]:
train

In [ ]:
train["label"].unique()

In [ ]:
train["label"].value_counts() #비율이 각각 다르기 때문에 validation셋 뽑을 때 주의해야함.val셋은 test셋의 비율을 따라가야함

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_valid = train_test_split(train, test_size=0.2, random_state=1234)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator()

train_generator = idg.flow_from_dataframe(x_train, x_col = "path", y_col = "label", target_size = (200,200), batch_size = 64)
valid_generator = idg.flow_from_dataframe(x_valid, x_col = "path", y_col = "label", target_size = (200,200), batch_size = 64)

In [ ]:
test = pd.DataFrame({"path": glob.glob("/kaggle/input/plant-seedlings-classification/test/*")})
test_generator = idg.flow_from_dataframe(test, x_col = "path", y_col = None, target_size = (200,200), batch_size = 64,
                                        shuffle = False, class_mode = None)

In [ ]:
train_generator.class_indices

In [ ]:
label_map = []
for x in train_generator.class_indices:
    label_map.append(x)

In [ ]:
label_map

In [ ]:
# from keras import Sequential
# from keras.layers import *
# from keras.applications.inception_v3 import InceptionV3

# model = Sequential()
# model.add(InceptionV3(weights="imagenet", include_top=False, pooling="avg"))
# model.add(Dense(12, activation="softmax"))
# model.compile(metrics = ["acc"], loss = "categorical_crossentropy", optimizer = "adam")

# model.fit(train_generator, validation_data=valid_generator, epochs=2)

# result = model.predict_classes(test_generator)
# # result = model.predict(test_generator)
# # result = result.argmax(axis=-1) 이 두줄이 위 한줄이랑 같은 결과

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from keras import Sequential
from keras.layers import *
from efficientnet.tfkeras import EfficientNetB1 # 이거 다음엔 EfficientNetB1로 하자 B0은 좀더 빠른데 성능은 약간 덜좋음
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau # epoch높게 적고 알아서 성능좋을 때 멈추게 # 이미지말고 다른 때도 다 쓰임

es = EarlyStopping(patience=3) #patience : val lose기준
mc = ModelCheckpoint("best.h5",save_best_only=True) #첫번째는 파일의 이름(아무거나해도됨)
rl = ReduceLROnPlateau(patience=2) #factor 기본값 0.1
# ReduceLROnPlateau : learning rate가 처음엔 좀 크다가 plateau에 가까워질수록 작아지는것

model = Sequential()
model.add(EfficientNetB1(weights="imagenet", include_top=False, pooling="avg")) # weights값을 imagenet에서 가져옴. transfer learning 전이학습(pre trained model)
#include_top(:맨마지막출력층)은 가져오지 않겠다(false)
model.add(Dense(12, activation="softmax"))
model.compile(metrics = ["acc"], loss = "categorical_crossentropy", optimizer = "adam") #여기 들어가는 옵션은 다음 시간에. accuracy

histoy = model.fit(train_generator, validation_data=valid_generator, epochs=1, callbacks=[es,mc,rl])


In [ ]:
model.load_weights("best.h5")
result = model.predict(test_generator)

result = result.argmax(axis=-1)

count = 0
for i in range(len(result)):
    if test['answer'][i] == result[i]:
        count += 1

accuracy = count / len(result)
print(accuracy)

In [ ]:
model.load_weights("best.h5") # 이거 빼먹지 않도록 주의!!
result = model.predict(test_generator)

In [ ]:
df = pd.DataFrame()

for i in range(len(result)):
    df = df.append(pd.Series(result[i], name=test["path"].apply(lambda x : x.split("/")[-1])[i]))

label_result = result.argmax(axis=-1)

df["label_raw"] = label_result

label_map = []
for x in train_generator.class_indices:
    label_map.append(x)

df['label']=df['label_raw'].apply(lambda x : label_map[x])

In [ ]:
result

In [ ]:
result = result.argmax(axis=-1)
result

In [ ]:
result[0]

In [ ]:
np.round_(result[2],3)

In [ ]:
result[1].sum()

In [ ]:
result = result.argmax(axis=-1)
result

In [ ]:
test

In [ ]:
test["answer"] = 1

In [ ]:
test

In [ ]:
count = 0
for i in range(len(result)):
    if test['answer'][i] == result[i]:
        count += 1

accuracy = count / len(result)
print(accuracy)

In [ ]:
sub = pd.read_csv("/kaggle/input/plant-seedlings-classification/sample_submission.csv")

In [ ]:
# pd.options.display.max_rows = 900

In [ ]:
label_map = []
for x in train_generator.class_indices:
    label_map.append(x)

sub["species"] = result
sub["species"] = sub["species"].apply(lambda x : label_map[x])
sub["file"] = test["path"]
sub["file"] = sub["file"].apply(lambda x : x.split("/")[5])
sub.to_csv("sub_1009.csv",index=0)

In [ ]:
sub["file"] = test["path"]

In [ ]:
sub

In [ ]:
sub["file"].apply(lambda x : x.split("/")[5])

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'acc']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

In [ ]:
plot_metrics(histoy)